In [1]:
import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
model = PhysNet(TN=True)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4), loss=np_loss)
model(np.random.random((4, 128, 32, 32, 3)));
model.summary()

Model: "phys_net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (4, 128, 32, 32, 16)   │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (4, 128, 16, 16, 32)   │        13,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (4, 128, 16, 16, 64)   │        55,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (4, 64, 8, 8, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (4, 64, 8, 8, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_5 (Sequential)       │ (4, 32, 4, 4, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_6 (Sequential)       │ (4, 32, 4, 4, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_7 (Sequential)       │ (4, 32, 2, 2, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_8 (Sequential)       │ (4, 32, 2, 2, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_9 (Sequential)       │ (4, 64, 2, 2, 64)      │        16,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_10 (Sequential)      │ (4, 128, 2, 2, 64)     │        16,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_9 (Conv3D)               │ (4, 128, 1, 1, 1)      │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (4, 128, 16, 16, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (4, 64, 8, 8, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling3d               │ (4, 128, 1, 1, 64)     │             0 │
│ (AveragePooling3D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (4, 128)               │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769,825 (2.94 MB)

 Trainable params: 768,577 (2.93 MB)

 Non-trainable params: 1,248 (4.88 KB)

In [3]:
batch_size = 128

rlap_tape = "/root/ssd_cache/rppg_training_data/rlap_128x32x32_all"

train_rlap = load_datatape(rlap_tape, fold='train', batch=batch_size)
valid_rlap = load_datatape(rlap_tape, fold='val', extended_hr='False', batch=batch_size)

#train_rlap = DatatapeMonitor(train_rlap)
training_set, validation_set = KerasDataset(train_rlap), KerasDataset(valid_rlap)

training_set = training_set.apply_fn(compress_aug)

rlap_128x32x32_all             datatape has been loaded.    33024 items total. fold=train selected
rlap_128x32x32_all             datatape has been loaded.     4096 items total. fold=val&extended_hr=False selected


In [4]:
#model.fit(training_set, validation_data=validation_set, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/physnettn.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(model, training_set, validation_set, epochs=20, check_point_path='../weights/physnettn.weights.h5')
model.load_weights('../weights/physnettn.weights.h5')

Training:   0%|          | 0/258 [00:00<?, ?it/s]

Validating:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch   1:  Training loss:0.4458	Validation loss:0.2601	


Training:   0%|          | 0/258 [00:00<?, ?it/s]

Validating:   0%|          | 0/32 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch  19:  Training loss:0.1281	Validation loss:0.1598	


Training:   0%|          | 0/258 [00:00<?, ?it/s]

Validating:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch  20:  Training loss:0.1265	Validation loss:0.1605	


In [5]:
model = PhysNet(TN=True)
model(np.random.random((4, 128, 32, 32, 3)));
model.load_weights('../weights/physnettn.weights.h5')

In [10]:
eval_on_dataset(dataset_H5_mmpd, pmodel(model), 128, (32, 32), step=1, batch=16, save='../results/PhysNetTN_RLAP_MMPD.h5')
get_metrics('../results/PhysNetTN_RLAP_MMPD.h5', dropped='False')

  0%|          | 0/660 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,10.994±0.182,18.999±3.11,12.163±0.189,0.37641
Whole,8.883±0.44,14.342±4.305,9.594±0.419,0.58398


In [7]:
eval_on_dataset(dataset_H5_cohface, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNetTN_RLAP_COHFACE.h5', fps=30)
get_metrics('../results/PhysNetTN_RLAP_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,3.089±0.165,7.681±2.446,4.202±0.211,0.78169
Whole,2.161±0.338,4.842±2.794,2.848±0.405,0.91142


In [8]:
eval_on_dataset(dataset_H5_pure, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNetTN_RLAP_PURE.h5')
get_metrics('../results/PhysNetTN_RLAP_PURE.h5')

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,0.532±0.09,2.54±1.906,0.746±0.107,0.99386
Whole,0.344±0.127,1.035±0.822,0.472±0.183,0.99909


In [9]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNetTN_RLAP_UBFC.h5')
get_metrics('../results/PhysNetTN_RLAP_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,1.232±0.084,2.285±1.027,1.266±0.092,0.99093
Whole,0.482±0.074,0.682±0.418,0.466±0.064,0.99931
